# MindDish.ai: Multimodal YouTube QA Bot

**Ironhack Final Project - Building a Multimodal AI ChatBot for YouTube Video QA**

This notebook implements a Retrieval-Augmented Generation (RAG) pipeline for cooking videos. The system retrieves YouTube video transcripts, processes them into embeddings, stores them in ChromaDB, and uses an LLM agent with 17 specialized tools to answer cooking questions.

**Features:**
- 28 curated cooking videos across 7 global cuisines
- 17 specialized cooking tools
- Multilingual support (Portuguese, Spanish, French, English)
- LangSmith integration for monitoring and evaluation
- 3-layer ingredient substitution safety system
- Tavily web search integration

## Step 0: Environment Setup and Imports

In [1]:
# Install required packages
%pip install -q langchain langchain-core langchain-openai langchain-community chromadb youtube-transcript-api python-dotenv tiktoken tavily-python

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Core imports
import os
import json
import re
import logging
from pathlib import Path
from typing import List, Dict, Any, Optional
from datetime import datetime

# LangChain imports
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, AIMessage

# Agent
from langchain.agents import create_agent

# Environment setup
from dotenv import load_dotenv
load_dotenv()

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Verify environment variables
print("Environment Check:")
print(f"OPENAI_API_KEY: {'Set' if os.getenv('OPENAI_API_KEY') else 'Missing'}")
print(f"LANGCHAIN_TRACING_V2: {os.getenv('LANGCHAIN_TRACING_V2')}")
print(f"LANGCHAIN_PROJECT: {os.getenv('LANGCHAIN_PROJECT')}")
print(f"LANGCHAIN_API_KEY: {'Set' if os.getenv('LANGCHAIN_API_KEY') else 'Missing'}")
print(f"TAVILY_API_KEY: {'Set' if os.getenv('TAVILY_API_KEY') else 'Missing'}")

Environment Check:
OPENAI_API_KEY: Set
LANGCHAIN_TRACING_V2: true
LANGCHAIN_PROJECT: MindDish_QABot
LANGCHAIN_API_KEY: Set
TAVILY_API_KEY: Set


In [3]:
import logging
logging.getLogger("langsmith").setLevel(logging.ERROR)

In [4]:
# Path configuration
DATA_DIR = Path("data")
TRANSCRIPTS_DIR = DATA_DIR / "transcripts"
CHROMA_DIR = DATA_DIR / "chroma_minddish"
CURATED_INDEX_PATH = DATA_DIR / "curated_index.json"

# Verify paths exist
print(f"Data directory exists: {DATA_DIR.exists()}")
print(f"Transcripts directory exists: {TRANSCRIPTS_DIR.exists()}")
if TRANSCRIPTS_DIR.exists():
    print(f"Transcript files found: {len(list(TRANSCRIPTS_DIR.glob('*.txt')))}")

Data directory exists: True
Transcripts directory exists: True
Transcript files found: 28


## Step 1: Curated Video Collection

In [5]:
# Curated video collection - 28 videos across 7 cuisines
curated_videos = {
    "african_cuisine": [
        {"id": "vIIyn8LH1_E", "title": "Nigerian Efo Riro", "url": "https://youtu.be/vIIyn8LH1_E"},
        {"id": "8ACc_oqhQRQ", "title": "Nigerian Moi-Moi", "url": "https://youtu.be/8ACc_oqhQRQ"},
        {"id": "6VXYzN_gDNs", "title": "Ghanaian Kontomire Stew", "url": "https://youtu.be/6VXYzN_gDNs"},
        {"id": "FiiTy8FpxqY", "title": "South African Chakalaka", "url": "https://youtu.be/FiiTy8FpxqY"},
    ],
    "french_cuisine": [
        {"id": "Q5uBEWjNeTw", "title": "Flognarde Dessert", "url": "https://youtu.be/Q5uBEWjNeTw"},
        {"id": "GFuBsSrIVaE", "title": "Coq au Vin", "url": "https://youtu.be/GFuBsSrIVaE"},
        {"id": "tOgH_ElyGQg", "title": "Poulet au Vinaigre", "url": "https://youtu.be/tOgH_ElyGQg"},
        {"id": "9qCO2qKrfr4", "title": "Ratatouille", "url": "https://youtu.be/9qCO2qKrfr4"},
    ],
    "portuguese_cuisine": [
        {"id": "xuUelAOuH3o", "title": "Bacalhau", "url": "https://youtu.be/xuUelAOuH3o"},
        {"id": "4sRwu9BnLAU", "title": "Carne Estufada", "url": "https://youtu.be/4sRwu9BnLAU"},
        {"id": "MA4LEjxZ7io", "title": "Pastel de Nata", "url": "https://youtu.be/MA4LEjxZ7io"},
        {"id": "2PoVTipLxoI", "title": "Bifanas", "url": "https://youtu.be/2PoVTipLxoI"},
    ],
    "jamaican_cuisine": [
        {"id": "d6IKVNRDjUk", "title": "Jamaican Pork Shoulder", "url": "https://youtu.be/d6IKVNRDjUk"},
        {"id": "QIG6weWWB4Q", "title": "Jamaican Curry Chicken", "url": "https://youtu.be/QIG6weWWB4Q"},
        {"id": "RlZx52eyW4M", "title": "Sweet and Sour Fish", "url": "https://youtu.be/RlZx52eyW4M"},
        {"id": "qHC2WBx8Cvg", "title": "Rice and Peas", "url": "https://youtu.be/qHC2WBx8Cvg"},
    ],
    "syrian_cuisine": [
        {"id": "tB5XsB91-fQ", "title": "Shrakiye", "url": "https://youtu.be/tB5XsB91-fQ"},
        {"id": "HMXByWj_TAY", "title": "Tabouleh", "url": "https://youtu.be/HMXByWj_TAY"},
        {"id": "DJD4QQkItT4", "title": "Knafe Nabulsieh", "url": "https://youtu.be/DJD4QQkItT4"},
        {"id": "xF4XRASGaC0", "title": "Fatteh", "url": "https://youtu.be/xF4XRASGaC0"},
    ],
    "italian_cuisine": [
        {"id": "bGJMHjG85BM", "title": "Chicken Cacciatore", "url": "https://youtu.be/bGJMHjG85BM"},
        {"id": "E--DfY3w15k", "title": "Cannelloni", "url": "https://youtu.be/E--DfY3w15k"},
        {"id": "WttUeyXPCbU", "title": "Zozzona", "url": "https://youtu.be/WttUeyXPCbU"},
        {"id": "llV1kYg5zNo", "title": "Gnocchi alla Sorrentina", "url": "https://youtu.be/llV1kYg5zNo"},
    ],
    "indian_cuisine": [
        {"id": "PRw88q0NkiY", "title": "Chana Masala", "url": "https://youtu.be/PRw88q0NkiY"},
        {"id": "nilVmkdmabs", "title": "Chilli Garlic Tawa Chicken", "url": "https://youtu.be/nilVmkdmabs"},
        {"id": "s6h3b4tuhCE", "title": "Coconut Dosa", "url": "https://youtu.be/s6h3b4tuhCE"},
        {"id": "wmbpOb9neLY", "title": "Garlic Naan Bread", "url": "https://youtu.be/wmbpOb9neLY"},
    ],
}

print(f"Curated cuisines: {len(curated_videos)}")
print(f"Total videos: {sum(len(v) for v in curated_videos.values())}")
print("\nVideos by cuisine:")
for cuisine, videos in curated_videos.items():
    print(f"  {cuisine}: {len(videos)} videos")

Curated cuisines: 7
Total videos: 28

Videos by cuisine:
  african_cuisine: 4 videos
  french_cuisine: 4 videos
  portuguese_cuisine: 4 videos
  jamaican_cuisine: 4 videos
  syrian_cuisine: 4 videos
  italian_cuisine: 4 videos
  indian_cuisine: 4 videos


## Step 2: Transcript Loading and Preprocessing

In [6]:
def clean_transcript_text(text: str) -> str:
    """Clean transcript text by removing timestamps, music markers, and noise."""
    if not text:
        return ""
    
    # Remove timestamp patterns
    text = re.sub(r'\d{1,2}:\d{2}(:\d{2})?', '', text)
    
    # Remove music/sound markers
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'\(.*?\)', '', text)
    
    # Clean whitespace
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    
    return text


def load_curated_index(path: Path = CURATED_INDEX_PATH) -> List[Dict[str, Any]]:
    """Load curated_index.json."""
    if not path.exists():
        logger.warning(f"Curated index not found at {path}")
        return []
    with path.open("r", encoding="utf-8") as f:
        return json.load(f)


def build_documents_from_index(index: List[Dict[str, Any]]) -> List[Document]:
    """Build LangChain Documents from curated index."""
    docs = []
    
    for entry in index:
        transcript_path = entry.get("transcript_path")
        if not transcript_path:
            continue
        
        text_path = Path(transcript_path)
        if not text_path.exists():
            continue
        
        raw_text = text_path.read_text(encoding="utf-8")
        cleaned_text = clean_transcript_text(raw_text)
        
        if not cleaned_text:
            continue
        
        metadata = {
            "collection": entry.get("collection", "unknown"),
            "title": entry.get("title", "Unknown"),
            "url": entry.get("url", ""),
            "video_id": entry.get("video_id", ""),
            "transcript_source": entry.get("transcript_source", "unknown"),
        }
        
        docs.append(Document(page_content=cleaned_text, metadata=metadata))
    
    return docs


# Load and build documents
curated_index = load_curated_index()
print(f"Loaded {len(curated_index)} entries from curated index")

if curated_index:
    raw_documents = build_documents_from_index(curated_index)
    print(f"Built {len(raw_documents)} documents with transcripts")

Loaded 28 entries from curated index
Built 28 documents with transcripts


## Step 3: Text Chunking and Vector Store Creation

In [7]:
# Initialize text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    separators=["\n\n", "\n", ". ", " ", ""],
)

# Chunk documents
if 'raw_documents' in dir() and raw_documents:
    chunked_documents = text_splitter.split_documents(raw_documents)
    print(f"Created {len(chunked_documents)} chunks from {len(raw_documents)} documents")
    
    if chunked_documents:
        print(f"\nSample chunk metadata: {chunked_documents[0].metadata}")
        print(f"Sample chunk content: {chunked_documents[0].page_content[:200]}...")

Created 225 chunks from 28 documents

Sample chunk metadata: {'collection': 'african_cuisine', 'title': 'Nigerian Efo Riro', 'url': 'https://youtu.be/vIIyn8LH1_E', 'video_id': 'vIIyn8LH1_E', 'transcript_source': 'local'}
Sample chunk content: This is the way I've been making efro now. It comes together very quickly. It tastes unique and delicious. It's not your typical efro. If you want to see how to make it, keep watching. Don't forget to...


In [8]:
# Initialize embeddings and LLM
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)

# Create or load ChromaDB vectorstore
CHROMA_DIR.mkdir(parents=True, exist_ok=True)

if 'chunked_documents' in dir() and chunked_documents:
    vectorstore = Chroma.from_documents(
        documents=chunked_documents,
        embedding=embeddings,
        persist_directory=str(CHROMA_DIR),
        collection_name="minddish_curated",
    )
    vectorstore.persist()
    print(f"Built and persisted vectorstore with {vectorstore._collection.count()} chunks")
else:
    vectorstore = Chroma(
        persist_directory=str(CHROMA_DIR),
        embedding_function=embeddings,
        collection_name="minddish_curated",
    )
    print(f"Loaded existing vectorstore with {vectorstore._collection.count()} chunks")

2025-11-29 18:44:37,900 - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
2025-11-29 18:44:39,216 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Built and persisted vectorstore with 450 chunks


/var/folders/2n/gs_200v517s5clp2y1jt0mgc0000gn/T/ipykernel_62175/2923697148.py:15: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


## Step 4: QA Chain and Indexed Videos

In [9]:
# Create retriever
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 4}
)

# Format retrieved documents
def format_docs(docs: List[Document]) -> str:
    formatted = []
    for doc in docs:
        source = f"[{doc.metadata.get('collection', 'unknown')}] {doc.metadata.get('title', 'Unknown')}"
        formatted.append(f"Source: {source}\nContent: {doc.page_content}")
    return "\n\n".join(formatted)

# QA prompt template
qa_template = """You are MindDish.ai, a cooking assistant that answers questions based ONLY on the provided video transcripts.

Context from cooking videos:
{context}

Question: {question}

Instructions:
- Answer ONLY based on the context provided
- If the information is not in the context, say "I don't have this information in the indexed videos"
- Always cite which video the information comes from
- Be helpful and provide step-by-step instructions when relevant

Answer:"""

qa_prompt = ChatPromptTemplate.from_template(qa_template)

# Build QA chain (rag_chain)
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | qa_prompt
    | llm
    | StrOutputParser()
)

print("RAG Chain created successfully")

RAG Chain created successfully


In [10]:
# Build indexed videos dictionary for tool lookups
indexed_videos = {}

if curated_index:
    for entry in curated_index:
        video_id = entry.get("video_id")
        if video_id:
            indexed_videos[video_id] = {
                "title": entry.get("title", "Unknown"),
                "collection": entry.get("collection", "unknown"),
                "url": entry.get("url", f"https://youtu.be/{video_id}"),
                "chunks": 0,
                "method": "youtube_transcript_api"
            }

print(f"Indexed {len(indexed_videos)} videos")

Indexed 28 videos


## Step 5: Web Search Integration (Tavily)

In [11]:
%pip install tavily-python

Note: you may need to restart the kernel to use updated packages.


In [12]:
def web_search(query: str, max_results: int = 3, search_depth: str = "basic") -> dict:
    """Search the web using Tavily API."""
    try:
        from tavily import TavilyClient
        
        tavily_api_key = os.getenv("TAVILY_API_KEY")
        
        if not tavily_api_key:
            return {
                "status": "error",
                "results": [],
                "message": "TAVILY_API_KEY not found"
            }
        
        client = TavilyClient(api_key=tavily_api_key)
        
        response = client.search(
            query=query,
            max_results=max_results,
            search_depth=search_depth,
        )
        
        formatted_results = []
        for result in response.get('results', []):
            formatted_results.append({
                'title': result.get('title', 'No title'),
                'url': result.get('url', ''),
                'content': result.get('content', ''),
                'score': result.get('score', 0.0)
            })
        
        return {
            "status": "success",
            "results": formatted_results,
            "query": query,
            "answer": response.get('answer', '')
        }
    
    except ImportError:
        return {"status": "error", "results": [], "message": "tavily-python not installed"}
    except Exception as e:
        return {"status": "error", "results": [], "message": f"Web search failed: {str(e)[:200]}"}

print("Web search function ready")

Web search function ready


## Step 6: Define All 17 Cooking Tools

In [13]:
# CORE RAG TOOLS (1-3)

@tool
def video_qa_tool(question: str) -> str:
    """Answer cooking questions using RAG across all indexed videos."""
    answer = rag_chain.invoke(question)
    return answer

@tool
def transcript_search_tool(keyword: str) -> str:
    """Search for specific keyword mentions across all cooking videos."""
    results = vectorstore.similarity_search(keyword, k=5)
    
    video_counts = {}
    for doc in results:
        video_title = doc.metadata.get('title', 'Unknown')
        video_counts[video_title] = video_counts.get(video_title, 0) + 1
    
    if not video_counts:
        return f"'{keyword}' not found in any videos"
    
    response = f"Found '{keyword}' in:\n"
    for video, count in sorted(video_counts.items(), key=lambda x: x[1], reverse=True):
        response += f"  - {video}: {count} mention(s)\n"
    
    return response

@tool
def list_videos_tool(query: str = "") -> str:
    """List all indexed cooking videos with details."""
    response = f"Indexed Cooking Videos ({len(indexed_videos)}):\n\n"
    for vid_id, info in indexed_videos.items():
        response += f"- {info['title']}\n"
        response += f"  ID: {vid_id} | Collection: {info['collection']}\n\n"
    return response

print("Core RAG tools defined (1-3)")

Core RAG tools defined (1-3)


In [14]:
# VIDEO ANALYSIS TOOLS (4-6)

@tool
def video_summary_tool(video_title_or_id: str) -> str:
    """Generate a comprehensive summary of a specific cooking video."""
    
    video_id = None
    for vid, info in indexed_videos.items():
        if video_title_or_id.lower() in info['title'].lower() or video_title_or_id == vid:
            video_id = vid
            break
    
    if not video_id:
        return f"Video '{video_title_or_id}' not found. Use list_videos_tool to see available videos."
    
    results = vectorstore.similarity_search(
        "recipe ingredients steps instructions",
        k=10,
        filter={"video_id": video_id}
    )
    
    if not results:
        return f"No content found for video {video_id}"
    
    all_text = " ".join([doc.page_content for doc in results[:8]])
    summary_prompt = f"""Summarize this cooking video including:
- Dish being made
- Main ingredients
- Key cooking steps
- Cooking time and techniques

Content: {all_text[:2000]}

Summary:"""
    
    summary = llm.invoke(summary_prompt)
    return summary.content

@tool
def compare_videos_tool(topic: str) -> str:
    """Compare how different cooking videos discuss a specific topic."""
    results = vectorstore.similarity_search(topic, k=8)
    
    video_content = {}
    for doc in results:
        video_title = doc.metadata.get('title', 'Unknown')
        if video_title not in video_content:
            video_content[video_title] = []
        video_content[video_title].append(doc.page_content)
    
    if not video_content:
        return f"No videos discuss '{topic}'"
    
    comparison = f"Comparison: '{topic}'\n\n"
    for video, chunks in video_content.items():
        excerpt = ' '.join(chunks[:2])[:200]
        comparison += f"{video}:\n   {excerpt}...\n\n"
    
    return comparison

@tool
def find_related_videos_tool(topic: str) -> str:
    """Find which cooking videos are most relevant to a topic."""
    results = vectorstore.similarity_search(topic, k=10)
    
    video_scores = {}
    for doc in results:
        video_title = doc.metadata.get('title', 'Unknown')
        video_scores[video_title] = video_scores.get(video_title, 0) + 1
    
    sorted_videos = sorted(video_scores.items(), key=lambda x: x[1], reverse=True)
    
    response = f"Most relevant videos for '{topic}':\n\n"
    for i, (video, score) in enumerate(sorted_videos[:3], 1):
        response += f"{i}. {video} ({score} relevant segments)\n"
    
    return response

print("Video analysis tools defined (4-6)")

Video analysis tools defined (4-6)


In [15]:
# RECIPE-SPECIFIC TOOLS (7-10)

@tool
def extract_ingredients_tool(video_title_or_id: str) -> str:
    """Extract all ingredients mentioned in a specific cooking video."""
    
    video_id = None
    for vid, info in indexed_videos.items():
        if video_title_or_id.lower() in info['title'].lower() or video_title_or_id == vid:
            video_id = vid
            break
    
    if not video_id:
        return f"Video '{video_title_or_id}' not found"
    
    results = vectorstore.similarity_search(
        "ingredients what you need shopping list",
        k=8,
        filter={"video_id": video_id}
    )
    
    if not results:
        return "No ingredients found"
    
    all_text = " ".join([doc.page_content for doc in results])
    prompt = f"""Extract all ingredients mentioned in this cooking video:

{all_text[:2000]}

Ingredients:"""
    
    response = llm.invoke(prompt)
    return response.content

@tool
def cooking_time_tool(video_title_or_id: str) -> str:
    """Extract cooking times, prep times, and total time from a video."""
    
    video_id = None
    for vid, info in indexed_videos.items():
        if video_title_or_id.lower() in info['title'].lower() or video_title_or_id == vid:
            video_id = vid
            break
    
    if not video_id:
        return f"Video '{video_title_or_id}' not found"
    
    results = vectorstore.similarity_search(
        "minutes hours time cook bake prep",
        k=6,
        filter={"video_id": video_id}
    )
    
    if not results:
        return "No timing information found"
    
    all_text = " ".join([doc.page_content for doc in results])
    prompt = f"""Extract all timing information from this cooking video:
- Prep time
- Cook time
- Total time

Content: {all_text[:1500]}

Time breakdown:"""
    
    response = llm.invoke(prompt)
    return response.content

@tool
def equipment_checker_tool(video_title_or_id: str) -> str:
    """List all cooking equipment and tools needed for a recipe."""
    
    video_id = None
    for vid, info in indexed_videos.items():
        if video_title_or_id.lower() in info['title'].lower() or video_title_or_id == vid:
            video_id = vid
            break
    
    if not video_id:
        return f"Video '{video_title_or_id}' not found"
    
    results = vectorstore.similarity_search(
        "pan pot skillet bowl oven stove equipment tools",
        k=6,
        filter={"video_id": video_id}
    )
    
    if not results:
        return "No equipment information found"
    
    all_text = " ".join([doc.page_content for doc in results])
    prompt = f"""List all cooking equipment and tools mentioned in this video:

Content: {all_text[:1500]}

Equipment needed:"""
    
    response = llm.invoke(prompt)
    return response.content

@tool
def smart_substitution_tool(original_ingredient: str, substitute: str, dish: str = "") -> str:
    """
    Suggest safe ingredient substitutions with 3-layer safety system:
    Layer 1: Dangerous substitutions blacklist
    Layer 2: Check indexed videos
    Layer 3: Web search with Tavily
    """
    
    # Layer 1: Dangerous substitutions blacklist
    dangerous_pairs = [
        ('baking soda', 'baking powder'),
        ('baking powder', 'baking soda'),
        ('salt', 'sugar'),
        ('sugar', 'salt'),
    ]
    
    ingredient_lower = original_ingredient.lower()
    substitute_lower = substitute.lower()
    
    for pair in dangerous_pairs:
        if any(item in ingredient_lower for item in pair) and any(item in substitute_lower for item in pair):
            return f"DANGER: Substituting {original_ingredient} with {substitute} is UNSAFE!"
    
    # Layer 2: Check indexed videos
    search_query = f"{original_ingredient} substitute {substitute} alternative"
    results = vectorstore.similarity_search(search_query, k=5)
    
    video_context = ""
    for doc in results:
        content = doc.page_content.lower()
        if ingredient_lower in content:
            video_context += doc.page_content + "\n"
    
    if video_context:
        prompt = f"""Can I use {substitute} instead of {original_ingredient} in {dish or 'this recipe'}?

Video context: {video_context[:800]}

Provide: YES/NO/DEPENDS, ratio, and how it affects the dish."""
        
        response = llm.invoke(prompt)
        return f"From indexed videos:\n{response.content}"
    
    # Layer 3: Web search
    web_result = web_search(f"substitute {original_ingredient} with {substitute} cooking", max_results=2)
    
    if web_result['status'] == 'success' and web_result['results']:
        web_context = "\n".join([r['content'][:200] for r in web_result['results']])
        prompt = f"""Can I substitute {original_ingredient} with {substitute}?

Web research: {web_context}

Provide clear YES/NO, ratio, and safety notes."""
        
        response = llm.invoke(prompt)
        return f"From web search:\n{response.content}"
    
    return f"No reliable information found for substituting {original_ingredient} with {substitute}."

print("Recipe-specific tools defined (7-10)")

Recipe-specific tools defined (7-10)


In [16]:
# VERIFICATION & ANALYSIS TOOLS (11-12)

@tool
def recipe_fact_check_tool(claim: str) -> str:
    """Verify cooking claims against indexed videos and web sources."""
    
    results = vectorstore.similarity_search(claim, k=4)
    
    if results:
        video_evidence = " ".join([doc.page_content for doc in results])
        prompt = f"""Fact-check this cooking claim:
Claim: {claim}

Video evidence: {video_evidence[:1000]}

Is this TRUE, FALSE, or PARTIALLY TRUE? Explain."""
        
        response = llm.invoke(prompt)
        return response.content
    
    return f"No evidence found in indexed videos for: {claim}"

@tool
def suggest_questions_tool(topic: str) -> str:
    """Suggest follow-up questions based on a cooking topic."""
    
    results = vectorstore.similarity_search(topic, k=3)
    
    context = " ".join([doc.page_content for doc in results]) if results else ""
    
    prompt = f"""Based on this cooking topic: {topic}

Context: {context[:500]}

Suggest 5 follow-up questions the user might want to ask:"""
    
    response = llm.invoke(prompt)
    return response.content

print("Verification & analysis tools defined (11-12)")

Verification & analysis tools defined (11-12)


In [17]:
# EXTERNAL & UTILITY TOOLS (13-17)

@tool
def web_search_cooking_tool(query: str) -> str:
    """Search the web for cooking information not in indexed videos."""
    
    result = web_search(query + " cooking recipe", max_results=3)
    
    if result['status'] == 'success' and result['results']:
        response = f"Web search results for '{query}':\n\n"
        for i, r in enumerate(result['results'], 1):
            response += f"{i}. {r['title']}\n   {r['content'][:150]}...\n   URL: {r['url']}\n\n"
        return response
    
    return f"No web results found for: {query}"

@tool
def cultural_context_tool(term: str) -> str:
    """Explain cultural context of cooking terms, dishes, or ingredients."""
    
    results = vectorstore.similarity_search(term, k=4)
    
    video_context = " ".join([doc.page_content for doc in results]) if results else ""
    
    if video_context:
        prompt = f"""Explain the cultural context of '{term}' based on this cooking video:

Video Context: {video_context[:500]}

Cultural Context:"""
        
        response = llm.invoke(prompt)
        video_answer = f"From videos:\n{response.content}\n\n"
    else:
        video_answer = f"'{term}' not found in indexed videos.\n\n"
    
    # Enhance with web search
    web_result = web_search(f"{term} traditional cooking cultural significance", max_results=2)
    
    if web_result['status'] == 'success' and web_result['results']:
        web_context = "\n".join([f"- {r['content'][:150]}" for r in web_result['results']])
        return video_answer + f"Additional context from web:\n{web_context}"
    
    return video_answer

@tool
def cooking_expert_analysis_tool(question: str) -> str:
    """Get detailed culinary analysis combining video content + web knowledge."""
    
    results = vectorstore.similarity_search(question, k=4)
    video_context = " ".join([doc.page_content for doc in results]) if results else "No relevant video content."
    
    web_result = web_search(question + " cooking science technique", max_results=2)
    web_context = "\n".join([r['content'][:200] for r in web_result.get('results', [])[:2]]) if web_result['status'] == 'success' else "Web unavailable."
    
    prompt = f"""As a culinary expert, analyze this question:

Question: {question}

Video Content: {video_context[:800]}

Web Research: {web_context[:400]}

Provide expert analysis covering technique, science, common mistakes, and pro tips."""
    
    response = llm.invoke(prompt)
    return response.content

@tool
def translate_recipe_tool(text: str, target_language: str = "Portuguese") -> str:
    """Translate cooking instructions or recipes to another language."""
    
    prompt = f"""Translate this cooking content to {target_language}. Maintain culinary terminology accuracy.

Text to translate:
{text[:1500]}

Translation in {target_language}:"""
    
    response = llm.invoke(prompt)
    return response.content

@tool
def nutrition_calculator_tool(video_title_or_recipe: str, servings: int = 1) -> str:
    """Calculate approximate nutritional information for a recipe."""
    
    try:
        video_id = None
        for vid, info in indexed_videos.items():
            if video_title_or_recipe.lower() in info['title'].lower():
                video_id = vid
                break
        
        if not video_id:
            return f"Video '{video_title_or_recipe}' not found."
        
        ingredients_text = extract_ingredients_tool.func(video_title_or_recipe)
        
        web_result = web_search(f"nutrition facts calories {video_title_or_recipe}", max_results=2)
        web_context = "\n".join([r['content'][:200] for r in web_result.get('results', [])[:2]]) if web_result['status'] == 'success' else "Web unavailable"
        
        prompt = f"""Calculate approximate nutritional information:

Recipe: {video_title_or_recipe}
Servings: {servings}

Ingredients: {ingredients_text[:1000]}

Web data: {web_context[:500]}

Provide: calories, protein, carbs, fat per serving. Note these are estimates."""
        
        response = llm.invoke(prompt)
        return f"Nutritional Information: {video_title_or_recipe}\nServings: {servings}\n\n{response.content}"
    
    except Exception as e:
        return f"Could not calculate nutrition: {str(e)[:200]}"

print("External & utility tools defined (13-17)")

External & utility tools defined (13-17)


## Step 7: Collect All Tools and Create Agent

In [18]:
# Collect all 17 tools
tools = [
    # Core RAG tools (1-3)
    video_qa_tool,
    transcript_search_tool,
    list_videos_tool,
    # Video analysis (4-6)
    video_summary_tool,
    compare_videos_tool,
    find_related_videos_tool,
    # Recipe-specific (7-10)
    extract_ingredients_tool,
    cooking_time_tool,
    equipment_checker_tool,
    smart_substitution_tool,
    # Verification (11-12)
    recipe_fact_check_tool,
    suggest_questions_tool,
    # External & utility (13-17)
    web_search_cooking_tool,
    cultural_context_tool,
    cooking_expert_analysis_tool,
    translate_recipe_tool,
    nutrition_calculator_tool,
]

print(f"MindDish.ai created with {len(tools)} specialized tools")
print("-" * 50)
print("\nTool Categories:")
print("  Core RAG: video_qa, transcript_search, list_videos")
print("  Video Analysis: video_summary, compare_videos, find_related")
print("  Recipe-Specific: ingredients, cooking_time, equipment, substitution")
print("  Verification: fact_check, suggest_questions")
print("  External: web_search, cultural_context, expert_analysis, translate, nutrition")

MindDish.ai created with 17 specialized tools
--------------------------------------------------

Tool Categories:
  Core RAG: video_qa, transcript_search, list_videos
  Video Analysis: video_summary, compare_videos, find_related
  Recipe-Specific: ingredients, cooking_time, equipment, substitution
  Verification: fact_check, suggest_questions
  External: web_search, cultural_context, expert_analysis, translate, nutrition


In [19]:
# Create agent using create_agent (LangChain 1.1.0+)
from langchain.agents import create_agent

system_prompt = """You are MindDish.ai, an expert cooking assistant with access to 28 curated cooking videos across 7 global cuisines (African, French, Portuguese, Jamaican, Syrian, Italian, Indian).

You have 17 specialized tools at your disposal. Use the appropriate tool for each task:
- For recipe questions: use video_qa_tool
- To find specific mentions: use transcript_search_tool
- To list available videos: use list_videos_tool
- For video summaries: use video_summary_tool
- To compare approaches: use compare_videos_tool
- For ingredient lists: use extract_ingredients_tool
- For cooking times: use cooking_time_tool
- For equipment needed: use equipment_checker_tool
- For substitutions: use smart_substitution_tool (has safety checks)
- To verify claims: use recipe_fact_check_tool
- For web searches: use web_search_cooking_tool
- For cultural context: use cultural_context_tool
- For expert analysis: use cooking_expert_analysis_tool
- To translate: use translate_recipe_tool
- For nutrition info: use nutrition_calculator_tool

Guidelines:
- Always cite which video your information comes from
- If information is not in indexed videos, say so and offer to search the web
- Be helpful, accurate, and provide step-by-step guidance when needed"""

# Create the agent graph
agent_graph = create_agent(
    model=llm,
    tools=tools,
    system_prompt=system_prompt
)

print(f"Agent created with {len(tools)} tools")

Agent created with 17 tools


## Step 8: Conversation Memory and Professional Handler

In [20]:
class ConversationManager:
    """Manage conversation history with the agent."""
    
    def __init__(self, agent_executor, max_history: int = 10):
        self.agent = agent_executor
        self.history = []
        self.max_history = max_history
    
    def chat(self, user_input: str) -> str:
        context = ""
        if self.history:
            recent = self.history[-self.max_history:]
            context = "\n".join([f"User: {h['user']}\nAssistant: {h['assistant'][:200]}" for h in recent])
            context = f"Previous conversation:\n{context}\n\n"
        
        full_input = context + user_input if context else user_input
        response = self.agent.invoke({"input": full_input})
        output = response.get("output", "")
        
        self.history.append({"user": user_input, "assistant": output})
        return output
    
    def clear_history(self):
        self.history = []


class RudenessDetector:
    """Detect rude or inappropriate messages."""
    
    def __init__(self):
        self.rude_words = ['stupid', 'dumb', 'idiot', 'useless', 'garbage', 'trash', 'terrible']
    
    def check(self, message: str) -> dict:
        message_lower = message.lower()
        rude_count = sum(1 for word in self.rude_words if word in message_lower)
        is_shouting = len(message) > 10 and message.isupper()
        
        return {
            "is_rude": rude_count > 0 or is_shouting,
            "severity": "high" if rude_count >= 2 or is_shouting else "medium" if rude_count == 1 else "none"
        }


class ProfessionalHandler:
    """Handle messages professionally."""
    
    def __init__(self):
        self.detector = RudenessDetector()
        self.responses = {
            "high": "I'm here to help with cooking questions. Let's have a constructive conversation.",
            "medium": "I notice some frustration. What cooking question can I help you with?"
        }
    
    def process(self, message: str) -> tuple:
        check = self.detector.check(message)
        if check["is_rude"]:
            return True, self.responses.get(check["severity"], self.responses["medium"])
        return False, message


conversation = ConversationManager(agent_graph)
handler = ProfessionalHandler()
print("Conversation manager and professional handler initialized")

Conversation manager and professional handler initialized


## Step 9: Testing and Evaluation

In [21]:
# Test the agent
test_queries = [
    "What videos do you have for Portuguese cuisine?",
    "How do I make Bifanas?",
    "What are the ingredients in Efo Riro?",
    "Can I substitute palm oil with vegetable oil?",
]

print("Agent Testing")
print("=" * 60)

for query in test_queries:
    print(f"\nQuery: {query}")
    print("-" * 40)
    
    is_rude, processed = handler.process(query)
    if is_rude:
        print(f"Response: {processed}")
        continue
    
    try:
        response = agent_graph.invoke({"messages": [("user", query)]})
        output = response["messages"][-1].content if response.get("messages") else "No response"
        print(f"Response: {output[:500]}..." if len(output) > 500 else f"Response: {output}")
    except Exception as e:
        print(f"Error: {e}")

Agent Testing

Query: What videos do you have for Portuguese cuisine?
----------------------------------------


2025-11-29 18:44:41,362 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-11-29 18:44:43,063 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Response: I have the following videos for Portuguese cuisine:
1. Bacalhau
2. Carne Estufada
3. Pastel de Nata
4. Bifanas

Let me know if you would like more information on any specific video!

Query: How do I make Bifanas?
----------------------------------------


2025-11-29 18:44:43,824 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-11-29 18:44:44,448 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-11-29 18:44:44,670 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-11-29 18:44:45,602 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-11-29 18:44:47,513 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Response: To make Bifanas, a Portuguese dish, you will need pork lard, onions, olive oil, and meat for the sandwiches. Here are the key steps involved in making Bifanas:

1. Pound out the meat to flatten it.
2. Make a sauce with onions and pork lard.
3. Assemble the sandwiches with the sauce and the pounded meat.

The video does not specify the exact cooking time, but the techniques involved include sautéing, pounding, and assembling the sandwiches. If you need more detailed instructions or specific quant...

Query: What are the ingredients in Efo Riro?
----------------------------------------


2025-11-29 18:44:48,213 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-11-29 18:44:48,996 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-11-29 18:44:53,247 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Response: I couldn't find the specific ingredients for Efo Riro in the indexed videos. However, I found some information from a web search:

Here are the ingredients typically used in Efo Riro:
- Sliced Spinach leaves
- Stock fish ear
- Assorted meat
- Sliced/ground tatashe
- Sliced pepper

You can find detailed recipes with these ingredients on websites like myactivekitchen.com, egunsifoods.com, and allnigerianfoods.com.

Query: Can I substitute palm oil with vegetable oil?
----------------------------------------


2025-11-29 18:44:53,877 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-11-29 18:44:54,142 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-11-29 18:44:54,827 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-11-29 18:44:55,605 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Response: Yes, you can substitute palm oil with vegetable oil in your recipe. You can use vegetable oil in equal amounts as a substitute for palm oil. Keep in mind that the flavor might be slightly different, but it should still work well in the dish.


## Step 9.5: LangSmith Tracing Verification

In [22]:
# Verify LangSmith is configured
print("LangSmith Configuration")
print("-" * 40)
print(f"LANGCHAIN_TRACING_V2: {os.getenv('LANGCHAIN_TRACING_V2')}")
print(f"LANGCHAIN_PROJECT: {os.getenv('LANGCHAIN_PROJECT')}")
print(f"LANGCHAIN_API_KEY: {'Configured' if os.getenv('LANGCHAIN_API_KEY') else 'Missing'}")

if os.getenv('LANGCHAIN_TRACING_V2') == 'true' and os.getenv('LANGCHAIN_API_KEY'):
    print("\nLangSmith tracing is ACTIVE")
    print("View traces at: https://smith.langchain.com/")
else:
    print("\nLangSmith tracing is NOT active")

LangSmith Configuration
----------------------------------------
LANGCHAIN_TRACING_V2: true
LANGCHAIN_PROJECT: MindDish_QABot
LANGCHAIN_API_KEY: Configured

LangSmith tracing is ACTIVE
View traces at: https://smith.langchain.com/


In [23]:
# Final test to generate LangSmith trace
print("Generating LangSmith trace...")
print("-" * 40)

response = agent_graph.invoke({"messages": [("user", "How do I make Nigerian Efo Riro?")]})
output = response["messages"][-1].content if response.get("messages") else "No response"
print(f"\nResponse: {output[:500]}...")

Generating LangSmith trace...
----------------------------------------


2025-11-29 18:44:56,899 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-11-29 18:44:57,802 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-11-29 18:44:58,192 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-11-29 18:44:59,289 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-11-29 18:45:01,145 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Response: To make Nigerian Efo Riro, a delicious soup, you will need spinach and a variety of spices. Here are the key steps involved in the cooking process:

1. Mix all the ingredients together to create a flavorful sauce.
2. Add in the spinach to the sauce.
3. Cook the dish for under 30 minutes.

For detailed instructions and additional tips, you can refer to the video titled "Nigerian Efo Riro" (ID: vIIyn8LH1_E). The recipe, along with many other Nigerian soup recipes, is available in a downloadable eb...


## Summary

This notebook implements a complete RAG-based cooking assistant with:

**Core Components:**
- YouTube transcript processing and cleaning
- ChromaDB vector store for semantic search
- QA chain for grounded responses
- Agent with 17 specialized cooking tools
- Conversation memory management
- Professional response handling
- Tavily web search integration

**Data:**
- 28 curated cooking videos
- 7 global cuisines: African, French, Portuguese, Jamaican, Syrian, Italian, Indian

**17 Tools:**
1. video_qa_tool
2. transcript_search_tool
3. list_videos_tool
4. video_summary_tool
5. compare_videos_tool
6. find_related_videos_tool
7. extract_ingredients_tool
8. cooking_time_tool
9. equipment_checker_tool
10. smart_substitution_tool
11. recipe_fact_check_tool
12. suggest_questions_tool
13. web_search_cooking_tool
14. cultural_context_tool
15. cooking_expert_analysis_tool
16. translate_recipe_tool
17. nutrition_calculator_tool

**Monitoring:**
- LangSmith integration for tracing and evaluation

**Deployment:**
- Live demo: https://minddish.ai
- API endpoint: https://api.minddish.ai